In [1]:
%matplotlib inline
import pandas as pd

In [3]:
compras_df = pd.read_csv('data/clean_compras.csv')

In [4]:
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Empezamos con una expansion de datos que tenemos, fecha por ejemplo

In [6]:
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])

In [7]:
compras_df['DIA'] = compras_df['FECHA'].apply(lambda x: x.day)

In [8]:
compras_df['MES'] = compras_df['FECHA'].apply(lambda x: x.month)

In [9]:
compras_df['ANO'] = compras_df['FECHA'].apply(lambda x: x.year)

## Proyectemos unas variables contra otras

In [15]:
beneficiario_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [16]:
list(beneficiario_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

In [18]:
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x: beneficiario_sales[x]) 

In [19]:
sheet_mean_compras = compras_df.groupby('SHEET')['IMPORTE'].mean()

In [20]:
sheet_mean_compras

SHEET
COMUNICACION              109705.343662
CONTRATISTAS Y FDO FED    474566.765787
GASTOS VARIOS             881659.731797
SERV PPROF                229592.166667
SERV. PERS.                59623.405217
Name: IMPORTE, dtype: float64

In [21]:
sheet_mean_compras.to_dict()

{'COMUNICACION': 109705.34366197184,
 'CONTRATISTAS Y FDO FED': 474566.7657872927,
 'GASTOS VARIOS': 881659.7317971003,
 'SERV PPROF': 229592.16666666666,
 'SERV. PERS.': 59623.40521739131}

In [22]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEET'].apply(lambda x: sheet_mean_compras[x]) 

## Traernos informacion externa, enriquecer los datos usando una tool de paginas de linkedin

In [24]:
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('data/unique_beneficiarios.csv', index=None)

In [26]:
#Cargar
linkedin_mty = pd.read_csv('data/linkedin_mty.csv')

In [29]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

In [31]:
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(lambda x: has_site[x.replace('  ', ' ')])

In [32]:
compras_df['HAS_LINKEDIN_SITE'] 

0       True
1       True
2       True
3       True
4       True
        ... 
1535    True
1536    True
1537    True
1538    True
1539    True
Name: HAS_LINKEDIN_SITE, Length: 1540, dtype: bool

In [33]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

True     1392
False     148
Name: HAS_LINKEDIN_SITE, dtype: int64

In [34]:
compras_df.to_csv('data/compras_df_enriched.csv', index=None)